In [1]:
import pandas as pd
import re

In [1]:
# Step 1: Load the dataset
df = pd.read_csv('D:/Music_Recommender/data/spotify_millsongdata.csv')

NameError: name 'pd' is not defined

In [3]:
len(df)

57650

In [4]:
df.columns

Index(['artist', 'song', 'link', 'text'], dtype='object')

In [5]:
# Step 2: Preprocess the lyrics in the 'text' column
def preprocess_lyrics(lyrics):
    lyrics = lyrics.lower()  # Convert to lowercase
    lyrics = re.sub(r'[^a-z\s]', '', lyrics)  # Remove special characters
    return lyrics

df['cleaned_lyrics'] = df['text'].apply(preprocess_lyrics)

In [6]:
# Step 3: Load NRC Emotion Lexicon
nrc_df = pd.read_csv('D:/Music_Recommender/data/nrc_lexicon.csv', encoding='latin1')  # Replace with your lexicon path
nrc_df = nrc_df[nrc_df['association'] == 1]

In [7]:
# Step 4: Define emotion mapping for your 7 categories
emotion_mapping = {
    'anger': 'angry',
    'joy': 'happy',
    'positive': 'happy',
    'anticipation': 'happy',
    'disgust': 'disgust',
    'fear': 'fear',
    'neutral': 'neutral',
    'sadness': 'sad',
    'negative': 'sad',
    'surprise': 'surprise'
}

In [8]:
# Step 5: Create an emotion dictionary from the NRC Lexicon
emotion_dict = {}
for _, row in nrc_df.iterrows():
    word = row['word']
    emotion = row['emotion']
    # Map the NRC emotion to one of the 7 target emotions
    target_emotion = emotion_mapping.get(emotion, 'neutral')
    if target_emotion not in emotion_dict:
        emotion_dict[target_emotion] = []
    emotion_dict[target_emotion].append(word)

# Step 6: Categorize songs based on the 'text' (lyrics) column
def categorize_song(lyrics):
    scores = {emotion: 0 for emotion in emotion_dict.keys()}
    for word in lyrics.split():
        for emotion, keywords in emotion_dict.items():
            if word in keywords:
                scores[emotion] += 1

    # Get the emotion with the highest score
    max_emotion = max(scores, key=scores.get)
    return max_emotion if max(scores.values()) > 0 else 'neutral'

df['emotion'] = df['cleaned_lyrics'].apply(categorize_song
                                           )

In [9]:
# Step 7: Filter the songs based on the emotion and keep relevant columns
happy_df = df[df['emotion'] == 'happy'][['artist','link', 'song','text']]
sad_df = df[df['emotion'] == 'sad'][['artist','link', 'song','text']]
surprise_df = df[df['emotion'] == 'surprise'][['artist','link', 'song','text']]
angry_df = df[df['emotion'] == 'angry'][['artist','link', 'song','text']]
disgust_df = df[df['emotion'] == 'disgust'][['artist','link', 'song','text']]
fear_df = df[df['emotion'] == 'fear'][['artist','link', 'song','text']]
neutral_df = df[df['emotion'] == 'neutral'][['artist','link', 'song','text']]

In [10]:

# Step 8: Save the results to separate CSV files
happy_df.to_csv('D:/Music_Recommender/data/songs/happy.csv', index=False)
sad_df.to_csv('D:/Music_Recommender/data/songs/sad.csv', index=False)
surprise_df.to_csv('D:/Music_Recommender/data/songs/surprise.csv', index=False)
angry_df.to_csv('D:/Music_Recommender/data/songs/angry.csv', index=False)
disgust_df.to_csv('D:/Music_Recommender/data/songs/disgust.csv', index=False)
fear_df.to_csv('D:/Music_Recommender/data/songs/fear.csv', index=False)
neutral_df.to_csv('D:/Music_Recommender/data/songs/neutral.csv', index=False)